# Tutti.ch Webscraper
## Mini Challenge 5 - wdb @ FHNW BSc Data Science
### Author: Lukas Reber

Script für Webscraping von tutti.ch mittels Selenium. Das Script sucht sämtliche Inserate welche mit dem definierten Begriff (Variable: searchterm, Zelle 256), gefunden werden und speichert folgende Attribute:
* Titel
* Beschreibung
* Kategorie
* Preis
* Anzahl Aufrufe
* Benutzername
* Datum des Inserats

Damit das Script funktioniert muss Chrome und der entsprechende Chrome Driver installiert sein

In [1]:
# using ChromeDriver 89

from selenium import webdriver
from selenium.webdriver.common.keys import Keys # used to send keys

# https://selenium-python.readthedocs.io/waits.html
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re
import pandas as pd

In [2]:
# path to chrome driver
PATH = './ChromeDriver/chromedriver'
driver = webdriver.Chrome(PATH)

# open the website
driver.get("https://tutti.ch")

# get title
print(driver.title)

Gratis Inserate in deiner Nähe - tutti.ch


In [3]:
searchterm = 'triptrapp'

# using the search bar
search = driver.find_element_by_name('search')
search.clear()
search.send_keys(searchterm)
search.send_keys(Keys.RETURN)

In [4]:
# getting the number of pages / search results
# this information is current not further used / saved

try:
    root = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "root"))
    )
    # find number of search results by xpath
    num_results = root.find_element_by_css_selector('h1')
    # for unknown reasons xpath contains doesnt work here: 
    # find_element_by_xpath("//*[contains(text(),'Inserate aus der ganzen Schweiz')]")
    n = re.findall(r'\d+',num_results.text)
    print(f'number of search results: {n[0]}')
except:
    driver.quit()

number of search results: 46


In [5]:
# getting all links
adlinks = []
condition = True
while condition:
    try:
        root = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "root"))
        )
        # find all titles by xpath search
        ads = root.find_elements_by_xpath('//a[@data-automation="ad-subject"]')
        for ad in ads:
            adlinks.append(ad.get_property('href'))

        # move to next page
        try:
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[2]/div/div[1]/div/div/div/div[1]/div/div[6]/nav/ul/li[6]/a'))
            )
            element.click()
        except:
            condition=False
    except:
        driver.quit()


    


In [269]:
# scraping all data

df = pd.DataFrame([], columns=['id','title','price','place','user','description','category','dateadded','views'])

for link in adlinks:
    driver.get(link)
    try:
        root = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "root"))
        )
        dict = {}
        dict['id'] = adlinks[0].split('/')[9]
        dict['title'] = root.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/div/div/div[1]/div[1]/div[1]/div/div[1]/div[1]/div[1]/h1').text
        dict['price'] = root.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/div/div/div[1]/div[1]/div[1]/div/div[1]/div[2]/div[1]/h2').text
        dict['place'] = root.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/div/div/div[1]/div[1]/div[1]/table/tbody/tr[4]/td/dd').text
        dict['user'] = root.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/div/div/div[1]/div[2]/div[2]/div[1]/div/div[2]/h4').text
        dict['description'] = root.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/div/div/div[1]/div[1]/div[1]/table/tbody/tr[2]/td').text
        dict['category'] = root.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/div/div/div[1]/div[1]/div[1]/table/tbody/tr[1]/td[1]/div/a').text
        dict['dateadded'] = root.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/div/div/div[1]/div[1]/div[1]/div/div[1]/div[2]/div[2]/div[1]/span').text
        dict['views'] = root.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/div/div/div[1]/div[1]/div[1]/div/div[1]/div[2]/div[2]/div[2]/span').text
        
        df = df.append(dict, ignore_index=True)

    except:
        driver.quit()

In [270]:
df

,id,title,price,place,user,description,category,dateadded,views
0,42145250,"Stokke BABYKISSEN (rosa, weiss gepunktet)",25.-,3043,Raspa,kein Einzelversand\nAbholung möglich\nVersand ...,Kinderwagen & Sitze,08.03.2021,36
1,42145250,"Stokke ""Babykissen"" für Tripp Trapp (Bio-BW, g...",28.-,3043,Raspa,Dieser Artikel ist RESERVIERT.\nVersand möglic...,Kinderwagen & Sitze,08.03.2021,8
2,42145250,"Stokke ""Babykissen"" zum Tripp Trapp Babyset (g...",23.-,3043,Raspa,Dieser Artikel ist RESERVIERT.\nVersand möglic...,Kinderwagen & Sitze,08.03.2021,13
3,42145250,2 Tripp Trapp kirsch & rot // NEUES MODELL/ Ve...,135.-,6000,Tiki,PREIS PRO STUHL OHNE ZUBEHÖR 135.-\n\nSehr sch...,Kinderwagen & Sitze,07.03.2021,110
4,42145250,Stokke Tripp Trapp mit Baby Set (natur- Newbor...,130.-,3043,Raspa,Foto ist ein Symbolfoto. Originale folgen...\n...,Kinderwagen & Sitze,06.03.2021,98
5,42145250,Stokke Tripp Trapp (plum purple - Newborn Set ...,130.-,3043,Raspa,Versand in demontiertem Zustand möglich (+ Fr....,Kinderwagen & Sitze,06.03.2021,38
6,42145250,"Stokke Tripp Trapp (red - Newborn Set passt, J...",120.-,3043,Raspa,Versand in demontiertem Zustand möglich (+ Fr....,Kinderwagen & Sitze,05.03.2021,49
7,42145250,"Stokke ""Tray"" (weiss)",30.-,3043,Raspa,"bevorzugt werden Käufer, die auch das Baby Set...",Kinderwagen & Sitze,05.03.2021,66
8,42145250,"Stokke ""Gurt"" zum Tripp Trapp (5-Punkt)",35.-,3043,Raspa,Dieser Artikel ist RESERVIERT.\nAbholung mögli...,Kinderwagen & Sitze,05.03.2021,29
9,42145250,"Stokke ""Holzbügel"" zum Tripp Trapp (rot)",40.-,3043,Raspa,Versand möglich (+ Fr. 7.-)\n\nZustand: gebrau...,Kinderwagen & Sitze,04.03.2021,32


In [6]:
# close the browser
driver.quit()